In [1]:
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


c:\ProgramData\Anaconda3\envs\8CC00\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from preprocessing import filter_df

df = filter_df()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
ALDH1_inhibition,,,,,,,,,,,,,,,,,,,,,
1,0.883077,0.732144,0.883077,0.001001,0.492906,0.688495,0.667061,0.688303,0.756098,0.0,...,0,0,0,0,1,0,0,0,0,0
1,0.807622,0.821655,0.807622,0.057684,0.466743,0.501554,0.501533,0.501222,0.500000,0.0,...,1,0,0,0,0,0,0,0,0,0
1,0.703186,0.828931,0.703186,0.014533,0.326910,0.719358,0.689464,0.719134,0.792683,0.0,...,0,0,0,0,1,0,0,0,0,0
1,0.757426,0.765366,0.757426,0.233129,0.823627,0.396428,0.405774,0.393233,0.256098,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.802564,0.359067,0.802564,0.074898,0.693622,0.591847,0.595718,0.591178,0.524390,0.0,...,0,1,0,0,0,1,0,1,0,0


In [9]:
df.to_csv("processed_data.csv", index=True)

In [3]:
df_filtered= df.reset_index()
#get label dataframe
y = df_filtered.iloc[:,0]
#get all features
X = df_filtered.loc[:, df_filtered.columns != 'ALDH1_inhibition']
#placeholder features and labels
# X = np.zeros((10,6))
# y = np.ones((10,1))

In [4]:
#create training set, validation set and test set 0.8:0.1:0.1 division
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size =0.5, random_state=1)

In [5]:
#join training features and labels for autogluon
train_set = pd.concat([X_train, y_train], axis=1)
#get label column name
label = 'ALDH1_inhibition'
#run algorithms on training set
predictor = TabularPredictor(label=label).fit(train_set)
val_set =  pd.concat([X_val, y_val], axis=1)
#shows leaderboard of all algorithms
predictor.leaderboard(val_set, silent=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20230613_074109\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230613_074109\"
AutoGluon Version:  0.7.0
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22000
Train Data Rows:    800
Train Data Columns: 208
Label Column: ALDH1_inhibition
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3749.84 MB
	Train Data (Original)  Memory Usage: 

	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 20 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Useless Original Features (Count: 16): ['NumRadicalElectrons', 'SMR_VSA8', 'SlogP_VSA9', 'EState_VSA11', 'fr_HOCCN', 'fr_azide', 'fr_benzodiazepine', 'fr_diazo', 'fr_dihydropyridine', 'fr_epoxide', 'fr_isocyan', 'fr_isothiocyan', 'fr_phos_acid', 'fr_phos_ester', 'fr_prisulfonamd', 'fr_term_acetylene']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 119 | ['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateInd

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.77,0.70625,0.013966,0.005985,0.837760,0.013966,0.005985,0.837760,1,True,4
1,ExtraTreesGini,0.75,0.70625,0.070810,0.049866,0.585920,0.070810,0.049866,0.585920,1,True,7
2,RandomForestGini,0.75,0.71875,0.116685,0.062831,0.756051,0.116685,0.062831,0.756051,1,True,5
3,LightGBMXT,0.74,0.70000,0.015957,0.005984,0.686418,0.015957,0.005984,0.686418,1,True,3
4,RandomForestEntr,0.74,0.71250,0.065825,0.056848,0.709103,0.065825,0.056848,0.709103,1,True,6
5,ExtraTreesEntr,0.74,0.71250,0.069813,0.050867,0.607045,0.069813,0.050867,0.607045,1,True,8
6,WeightedEnsemble_L2,0.74,0.75000,0.110702,0.059841,3.871745,0.016954,0.000998,0.544545,2,True,11
7,LightGBMLarge,0.73,0.70625,0.013964,0.006981,2.994516,0.013964,0.006981,2.994516,1,True,10
8,XGBoost,0.72,0.72500,0.022938,0.008977,2.741280,0.022938,0.008977,2.741280,1,True,9
9,KNeighborsDist,0.69,0.69375,0.167371,0.142618,0.029920,0.167371,0.142618,0.029920,1,True,2


In [6]:
predictor.predict(val_set, model='LightGBM')
y_true = val_set[label]  # true label
y_pred = predictor.predict(val_set)  # predicted label
confusion_matrix(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average="weighted")
print(recall)
print(precision)

0.74
0.6930434782608696
